In [1]:
"""
solving pendulum using actor-critic model
"""

import gym
import numpy as np
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Dropout, Input, GRU, Masking
from tensorflow.keras.layers import Add, Concatenate
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

import tensorflow as tf

import random
from collections import deque

def stack_samples(samples):
	array = np.array(samples)
	
	current_states = np.stack(array[:,0]).reshape((array.shape[0],-1,array[0,0].shape[2]))
	actions = np.stack(array[:,1]).reshape((array.shape[0],-1))
	rewards = np.stack(array[:,2]).reshape((array.shape[0],-1))
	new_states = np.stack(array[:,3]).reshape((array.shape[0],-1,array[0,3].shape[2]))
	dones = np.stack(array[:,4]).reshape((array.shape[0],1))
	
	return current_states, actions, rewards, new_states, dones

# determines how to assign values to each state, i.e. takes the state
# and action (two-input model) and determines the corresponding value
class ActorCritic:
	def __init__(self, env, sess):
		self.env  = env
		self.sess = sess

		self.learning_rate = 0.0001
		self.epsilon = .9
		self.epsilon_decay = .99995
		self.gamma = .90
		self.tau   = .01

		# ===================================================================== #
		#                               Actor Model                             #
		# Chain rule: find the gradient of chaging the actor network params in  #
		# getting closest to the final value network predictions, i.e. de/dA    #
		# Calculate de/dA as = de/dC * dC/dA, where e is error, C critic, A act #
		# ===================================================================== #
		self.ema = tf.train.ExponentialMovingAverage(decay=1-self.tau)
		self.memory = deque(maxlen=4000)
		self.actor_state_input, self.actor_model = self.create_actor_model()
		_, self.target_actor_model = self.create_actor_model()

		self.actor_critic_grad = tf.placeholder(tf.float32,
			[None, self.env.action_space.shape[0]]) # where we will feed de/dC (from critic)

		actor_model_weights = self.actor_model.trainable_weights
		self.actor_grads = tf.gradients(self.actor_model.output,
			actor_model_weights, -self.actor_critic_grad) # dC/dA (from actor)
		grads = zip(self.actor_grads, actor_model_weights)
		self.optimize = tf.train.AdamOptimizer(self.learning_rate).apply_gradients(grads)

		# ===================================================================== #
		#                              Critic Model                             #
		# ===================================================================== #

		self.critic_state_input, self.critic_action_input, \
			self.critic_model = self.create_critic_model()
		_, _, self.target_critic_model = self.create_critic_model()

		self.critic_grads = tf.gradients(self.critic_model.output,
			self.critic_action_input) # where we calcaulte de/dC for feeding above

		self.target_actor_upd = self.ema.apply(self.actor_model.trainable_weights)
		self.target_critic_upd = self.ema.apply(self.critic_model.trainable_weights)
		# Initialize for later gradient calculations
		self.sess.run(tf.initialize_all_variables())
        
        

	# ========================================================================= #
	#                              Model Definitions                            #

	def create_actor_model(self):
	# ========================================================================= #
		state_input = Input(shape=(None,self.env.observation_space.shape[0]))
		mask_state_input = Masking(mask_value=0.)(state_input)
		h1 = Dense(500, activation='relu')(mask_state_input)
		actor_rnn,state_h = GRU(256, return_state=True)(h1)
		h2 = Dense(500, activation='relu')(state_h)
		output = Dense(self.env.action_space.shape[0], activation='tanh')(h2)

		model = Model([state_input], output)
		adam  = Adam(lr=0.0001)
		model.compile(loss="mse", optimizer=adam)
		return state_input, model

	def create_critic_model(self):
		state_input = Input(shape=(None,self.env.observation_space.shape[0]))
		mask_state_input = Masking(mask_value=0.)(state_input)
		state_h1 = Dense(500, activation='relu')(mask_state_input)
		critic_rnn,state_h2 = GRU(256, return_state=True)(state_h1)

		action_input = Input(shape=self.env.action_space.shape)
		action_h1    = Dense(500)(action_input)

		merged    = Concatenate()([state_h2, action_h1])
		merged_h1 = Dense(500, activation='relu')(merged)
		output = Dense(1, activation='linear')(merged_h1)
		model  = Model([state_input,action_input],output)

		adam  = Adam(lr=0.0001)
		model.compile(loss="mse", optimizer=adam)
		return state_input, action_input, model

	# ========================================================================= #
	#                               Model Training                              #
	# ========================================================================= #

	def remember(self, cur_state, action, reward, new_state, done):
		self.memory.append([cur_state, action, reward, new_state, done])

	def _train_actor(self, samples):
		
			cur_states, actions, rewards, new_states, _ =  stack_samples(samples)
			predicted_actions = self.actor_model.predict(cur_states)
			grads = self.sess.run(self.critic_grads, feed_dict={
				self.critic_state_input:  cur_states,
				self.critic_action_input: predicted_actions
			})[0]

			self.sess.run(self.optimize, feed_dict={
				self.actor_state_input: cur_states,
				self.actor_critic_grad: grads
			})

	def _train_critic(self, samples):
   

		cur_states, actions, rewards, new_states, dones = stack_samples(samples)
		target_actions = self.target_actor_model.predict(new_states)
		future_rewards = self.target_critic_model.predict([new_states, target_actions])
		dones = dones.reshape(rewards.shape)
		future_rewards = future_rewards.reshape(rewards.shape)
		rewards += self.gamma * future_rewards * (1 - dones)
		
		evaluation = self.critic_model.fit([cur_states, actions], rewards, verbose=0)
		#print(evaluation.history)
        
	def train(self):
		batch_size = 256
		if len(self.memory) < batch_size:
			return

		rewards = []
		samples = random.sample(self.memory, batch_size)
		self.samples = samples
		self._train_critic(samples)
		self._train_actor(samples)

	# ========================================================================= #
	#                         Target Model Updating                             #
	# ========================================================================= #

	def _update_actor_target(self):
		target_actor_model_weights = self.target_actor_model.trainable_weights
		self.sess.run(self.target_actor_upd)
		target_actor_model_weights = [self.ema.average(x) for x in self.actor_model.trainable_weights]

	def _update_critic_target(self):
		target_critic_model_weights = self.target_critic_model.trainable_weights
		self.sess.run(self.target_critic_upd)
		target_critic_model_weights = [self.ema.average(x) for x in self.critic_model.trainable_weights]

	def update_target(self):
		self._update_actor_target()
		self._update_critic_target()

	# ========================================================================= #
	#                              Model Predictions                            #
	# ========================================================================= #

	def act(self, cur_state):
		self.epsilon *= self.epsilon_decay
		if np.random.random() < self.epsilon:
			return self.actor_model.predict(cur_state)*2 + np.random.normal()
		return self.actor_model.predict(cur_state)*2





/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [2]:
def main():
	sess = tf.Session()
	K.set_session(sess)
	env = gym.make("Pendulum-v0")
	actor_critic = ActorCritic(env, sess)

	num_trials = 10000
	trial_len  = 200

	for i in range(num_trials):
		print("trial:" + str(i))
		cur_state = env.reset()
		action = env.action_space.sample()
		reward_sum = 0
		obs_list = []        
		obs_list.append(cur_state)
		for j in range(trial_len):
			#env.render()        
			obs_seq = np.asarray(obs_list)
			obs_seq = obs_seq.reshape((1, -1, env.observation_space.shape[0]))
			action = actor_critic.act(obs_seq)
			action = action.reshape((1, env.action_space.shape[0]))

			new_state, reward, done, _ = env.step(action)
			reward += reward
			if j == (trial_len - 1):
				done = True
				print(reward)

			if (j % 5 == 0):
				actor_critic.train()
				actor_critic.update_target()   
			
			new_state = new_state.reshape((env.observation_space.shape))

			obs_list.append(new_state)
			next_obs_seq = np.asarray(obs_list)
			next_obs_seq = next_obs_seq.reshape((1, -1, env.observation_space.shape[0]))
            
			#padding
			pad_width = trial_len-np.size(obs_seq,1)
			obs_seq = np.pad(obs_seq,((0,0),(pad_width,0),(0,0)))
			next_obs_seq = np.pad(next_obs_seq,((0,0),(pad_width,0),(0,0)))
			#print(obs_seq.shape)
			#print(next_obs_seq.shape)
            
			actor_critic.remember(obs_seq, action, reward, next_obs_seq, done)
			cur_state = new_state

		if (i % 5 == 0) and i!=0:
			cur_state = env.reset()
			obs_list = []                   
			for j in range(500):
				#env.render()
				cur_state = cur_state.reshape((1, env.observation_space.shape[0]))
				obs_list.append(cur_state)
				obs_seq = np.asarray(obs_list)
				obs_seq = obs_seq.reshape((1, -1, env.observation_space.shape[0]))
				action = actor_critic.act(obs_seq)
				action = action.reshape((1, env.action_space.shape[0]))

				new_state, reward, done, _ = env.step(action)
				#reward += reward
				#if j == (trial_len - 1):
					#done = True
					#print(reward)

				#if (j % 5 == 0):
				#    actor_critic.train()
				#    actor_critic.update_target()   
				
				new_state = new_state.reshape((1, env.observation_space.shape[0]))

				#actor_critic.remember(cur_state, action, reward, new_state, done)
				cur_state = new_state
				

if __name__ == "__main__":
	main()

/usr/local/lib/python3.6/dist-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Instructions for updating:
Use `tf.global_variables_initializer` instead.
trial:0
[-17.977266]
trial:1
[-16.840416]
trial:2


KeyboardInterrupt: 